<a href="https://colab.research.google.com/github/PriyaBansal68/ML-Project-winter-2022/blob/main/xgboost_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
submit = True
search = False
estimators = 20
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Football/train.csv')

Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:

display(df.head())
df.shape
for i in df.columns:
    print(i)

,id,target,home_team_name,away_team_name,match_date,league_name,league_id,is_cup,home_team_coach_id,away_team_coach_id,...,away_team_history_league_id_1,away_team_history_league_id_2,away_team_history_league_id_3,away_team_history_league_id_4,away_team_history_league_id_5,away_team_history_league_id_6,away_team_history_league_id_7,away_team_history_league_id_8,away_team_history_league_id_9,away_team_history_league_id_10
0,11906497,away,Newell's Old Boys,River Plate,01-12-2019 00:45,Superliga,636,False,468196.0,468200.0,...,1122.0,642.0,636.0,636.0,636.0,1122.0,636.0,642.0,636.0,1122.0
1,11984383,home,Real Estelí,Deportivo Las Sabanas,01-12-2019 01:00,Primera Division,752,False,516788.0,22169161.0,...,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0,752.0
2,11983301,draw,UPNFM,Marathón,01-12-2019 01:00,Liga Nacional,734,False,2510608.0,456313.0,...,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0,734.0
3,11983471,away,León,Morelia,01-12-2019 01:00,Liga MX,743,False,1552508.0,465797.0,...,743.0,743.0,743.0,743.0,743.0,743.0,743.0,743.0,746.0,743.0
4,11883005,home,Cobán Imperial,Iztapa,01-12-2019 01:00,Liga Nacional,705,False,429958.0,426870.0,...,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0,705.0


id
target
home_team_name
away_team_name
match_date
league_name
league_id
is_cup
home_team_coach_id
away_team_coach_id
home_team_history_match_date_1
home_team_history_match_date_2
home_team_history_match_date_3
home_team_history_match_date_4
home_team_history_match_date_5
home_team_history_match_date_6
home_team_history_match_date_7
home_team_history_match_date_8
home_team_history_match_date_9
home_team_history_match_date_10
home_team_history_is_play_home_1
home_team_history_is_play_home_2
home_team_history_is_play_home_3
home_team_history_is_play_home_4
home_team_history_is_play_home_5
home_team_history_is_play_home_6
home_team_history_is_play_home_7
home_team_history_is_play_home_8
home_team_history_is_play_home_9
home_team_history_is_play_home_10
home_team_history_is_cup_1
home_team_history_is_cup_2
home_team_history_is_cup_3
home_team_history_is_cup_4
home_team_history_is_cup_5
home_team_history_is_cup_6
home_team_history_is_cup_7
home_team_history_is_cup_8
home_team_history_is_cup

In [26]:
data=df.drop(['match_date','home_team_history_match_date_1','league_name',
 'home_team_history_match_date_2',
 'home_team_history_match_date_3',
 'home_team_history_match_date_4',
 'home_team_history_match_date_5',
 'home_team_history_match_date_6',
 'home_team_history_match_date_7',
 'home_team_history_match_date_8',
 'home_team_history_match_date_9',
 'home_team_history_match_date_10', 
 'away_team_history_match_date_1',
 'away_team_history_match_date_2',
 'away_team_history_match_date_3',
 'away_team_history_match_date_4',
 'away_team_history_match_date_5',
 'away_team_history_match_date_6',
 'away_team_history_match_date_7',
 'away_team_history_match_date_8',
 'away_team_history_match_date_9','away_team_history_match_date_10'], axis=1)

In [27]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110938 entries, 0 to 110937
Columns: 192 entries, id to home_streak
dtypes: float64(162), int64(4), object(26)
memory usage: 162.5+ MB


In [28]:
#what is the win rate for the home team?

# Total number of matches.
n_matches = data.shape[0]

# Calculate number of features. -1 because we are saving one as the target variable (win/lose/draw)
n_features = data.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(data[data.target == 'home'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100

# Print the results
print ("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))
print ("Number of matches won by home team: {}".format(n_homewins))
print ("Win rate of home team: {:.2f}%".format(win_rate))

Total number of matches: 110938
Number of features: 169
Number of matches won by home team: 48113
Win rate of home team: 43.37%


In [29]:
data=data.dropna()

In [31]:
data['target'] = data['target'].replace(['away','draw','home'],[0, 1, 2])

In [32]:
def add_streak(df):
    h_t_goal = "home_team_history_goal_"
    h_t_opp_goal = "home_team_history_opponent_goal_"
    a_t_goal = "away_team_history_goal_"
    a_t_opp_goal = "away_team_history_opponent_goal_"
    df["away_streak"] = np.nan
    df["home_streak"] = np.nan
    def apply_streaks(x):
        home_streak = 0
        away_streak = 0
        home_streak_over = False
        away_streak_over = False
        
        if  x[h_t_goal+str(1)] > x[h_t_opp_goal+str(1)]:
            home_win_streak = True
        else:
            home_win_streak = False
            
        if  x[a_t_goal+str(1)] > x[a_t_opp_goal+str(1)]:
            away_win_streak = True
        else:
            away_win_streak = False
            
        def check_streak(streak,streak_over,win_streak,team_goal,opp_goal):
            if not streak_over and win_streak and team_goal > opp_goal:
                streak+=1
            elif not streak_over and not win_streak and team_goal < opp_goal:
                streak-=1
            elif not streak_over and team_goal == opp_goal:
                pass
            else:
                streak_over = True
            return streak, streak_over
                
        for i in range(1,11):
            if not home_streak_over:
                home_streak, home_streak_over = check_streak(home_streak,home_streak_over, home_win_streak, x[h_t_goal+str(i)],x[h_t_opp_goal+str(i)])
            if not away_streak_over:
                away_streak, away_streak_over = check_streak(away_streak,away_streak_over, away_win_streak, x[a_t_goal+str(i)],x[a_t_opp_goal+str(i)])
            
        return pd.Series([home_streak,away_streak],index=['home_streak','away_streak'])

    
    df[['home_streak','away_streak']] = df.apply(lambda x: apply_streaks(x), axis=1)
    return df

In [7]:
pip install category_encoders

     |████████████████████████████████| 86 kB 3.2 MB/s 


In [33]:

import category_encoders as ce
from sklearn.model_selection import train_test_split

X_train1 = add_streak(df)
X_train = X_train1.drop(['target'], axis=1)
y_train = X_train1['target']


#X_test = add_streak(X_test)

#if not submit:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

encoder = ce.OrdinalEncoder(cols=['home_team_name' , 'away_team_name', 'is_cup'])
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)
temp_df = X_test
for i in X_train.columns:
    if X_train[i].dtype == 'object':
        X_train = X_train.drop([i],axis=1)
        X_test = X_test.drop([i],axis=1)
    if "is_cup" in i or "history_is_play_home" in i:
        try:
            X_train = X_train.drop([i],axis=1)
            X_test = X_test.drop([i],axis=1)
        except:
            continue

for i in X_train.columns:
    if 'rating' not in i and 'goal':
        X_train[i] = X_train[i].fillna(-1)
        X_test[i] = X_test[i].fillna(-1)
    else:
        X_train[i] = X_train[i].fillna(X_train[i].mode())
        X_test[i] = X_test[i].fillna(X_train[i].mode())
lag_size = 9

In [34]:

for i in X_train.columns:
    print(i)

id
home_team_name
away_team_name
league_id
home_team_coach_id
away_team_coach_id
home_team_history_goal_1
home_team_history_goal_2
home_team_history_goal_3
home_team_history_goal_4
home_team_history_goal_5
home_team_history_goal_6
home_team_history_goal_7
home_team_history_goal_8
home_team_history_goal_9
home_team_history_goal_10
home_team_history_opponent_goal_1
home_team_history_opponent_goal_2
home_team_history_opponent_goal_3
home_team_history_opponent_goal_4
home_team_history_opponent_goal_5
home_team_history_opponent_goal_6
home_team_history_opponent_goal_7
home_team_history_opponent_goal_8
home_team_history_opponent_goal_9
home_team_history_opponent_goal_10
home_team_history_rating_1
home_team_history_rating_2
home_team_history_rating_3
home_team_history_rating_4
home_team_history_rating_5
home_team_history_rating_6
home_team_history_rating_7
home_team_history_rating_8
home_team_history_rating_9
home_team_history_rating_10
home_team_history_opponent_rating_1
home_team_history_op

In [35]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss,accuracy_score,roc_auc_score
#if not search:
best_hyperparams = {'colsample_bytree': 0.9080289013373745, 'gamma': 1.2419885617337658, 'max_depth': 8.0, 'min_child_weight': 4.0, 'n_estimators': 238.0, 'reg_alpha': 41.0, 'reg_lambda': 0.05050398016408508}
#if not submit and not search:
rfc = XGBClassifier(n_estimators=int(best_hyperparams['n_estimators']), max_depth = int(best_hyperparams['max_depth']), gamma = best_hyperparams['gamma'],
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree']))
    #fc = XGBClassifier(n_estimators=estimators)
rfc.fit(X_train, y_train)
y_pred = rfc.predict_proba(X_test)
y_pred_acc = rfc.predict(X_test)

print(y_test.values)
print(y_pred)
print('Log loss score with  : {0:0.4f}'. format(log_loss(y_test.values,y_pred)))
print('Model accuracy score with : {0:0.4f}'. format(accuracy_score(y_test, y_pred_acc)))
print('Model ROC with : {0:0.4f}'. format(roc_auc_score(y_test.values, y_pred,multi_class='ovr',average='weighted')))

['home' 'home' 'home' ... 'draw' 'draw' 'home']
[[0.17343359 0.21964292 0.6069235 ]
 [0.19572376 0.29082045 0.5134558 ]
 [0.31180665 0.2526416  0.43555173]
 ...
 [0.42134994 0.22238876 0.3562613 ]
 [0.29651678 0.23886186 0.4646214 ]
 [0.23511036 0.30080095 0.46408868]]
Log loss score with  : 1.0275
Model accuracy score with : 0.4900
Model ROC with : 0.6380


In [36]:
#if not submit and not search:
feature_scores = pd.Series(rfc.feature_importances_, index=X_train.columns).sort_values(ascending=False)

for i,v in feature_scores.items():
    print(i, v)

home_team_history_rating_1 0.025265594944357872
home_team_history_rating_2 0.02453991398215294
home_team_history_rating_3 0.023833267390727997
home_streak 0.02207372337579727
home_team_history_opponent_rating_2 0.02111603319644928
away_team_history_rating_2 0.018508141860365868
home_team_history_rating_4 0.018113790079951286
home_team_history_opponent_rating_6 0.01796553283929825
away_team_history_rating_5 0.01748504862189293
away_team_history_rating_1 0.017069008201360703
away_team_history_opponent_rating_1 0.016929782927036285
away_team_history_opponent_rating_5 0.016923563554883003
home_team_history_opponent_rating_8 0.016314759850502014
home_team_history_opponent_rating_5 0.016296548768877983
away_team_history_opponent_rating_2 0.01556048821657896
away_streak 0.015299231745302677
home_team_history_rating_5 0.01491444744169712
away_team_history_rating_3 0.01450531929731369
home_team_history_rating_9 0.014470960944890976
away_team_history_rating_8 0.0139570077881217
home_team_history

In [37]:
print(best_hyperparams)

{'colsample_bytree': 0.9080289013373745, 'gamma': 1.2419885617337658, 'max_depth': 8.0, 'min_child_weight': 4.0, 'n_estimators': 238.0, 'reg_alpha': 41.0, 'reg_lambda': 0.05050398016408508}


In [ ]:
"""from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
n_estimators = range(50, 400, 50)

# grid search
model = XGBClassifier()
param_grid = dict(n_estimators=n_estimators)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]
print()
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
"""

'from sklearn.model_selection import GridSearchCV\nfrom sklearn.model_selection import StratifiedKFold\nkfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)\nn_estimators = range(50, 400, 50)\n\n# grid search\nmodel = XGBClassifier()\nparam_grid = dict(n_estimators=n_estimators)\ngrid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)\ngrid_result = grid_search.fit(X_train, y_train)\nprint("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))\nmeans = grid_result.cv_results_["mean_test_score"]\nstds = grid_result.cv_results_["std_test_score"]\nparams = grid_result.cv_results_["params"]\nprint()\nfor mean, stdev, param in zip(means, stds, params):\n    print("%f (%f) with: %r" % (mean, stdev, param))\n'

In [ ]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[
        ("classifier", XGBClassifier(n_estimators=int(best_hyperparams['n_estimators']), max_depth = int(best_hyperparams['max_depth']), gamma = best_hyperparams['gamma'],
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree'])))])

pipeline.fit(X_train, y_train)
predictions = pipeline.predict_proba(X_test)
submission = pd.DataFrame(predictions ,columns=pipeline.classes_)

    
submission['id'] = temp_df['id']
"""for i in ['home','away','draw']:
        #submission.iloc[submission[[i]].max(axis=1)>0.5, i]
        submission.loc[(submission[['home','away','draw']].max(axis=1)>0.5) & (submission[['home','away','draw']].idxmax(axis=1).values == i),i] = 1
        submission.loc[(submission[['home','away','draw']].max(axis=1)>0.5) & (submission[['home','away','draw']].idxmax(axis=1).values != i),i] = 0"""
display(submission)
    #submit!
submission.to_csv('submission.csv', index=False)
